### Testing libraly
this will show us if we can use r inside the notebook

In [6]:
library(openxlsx)

file_path <- "/home/me/Desktop/Codin/zero_day/I-am-R/StudentPerformance/Health Facilities Cleanning.xlsx"

# Load workbook
wb <- loadWorkbook(file_path)

# Read data
hc_data <- readWorkbook(wb, sheet = "TelMentorshipFacilities")
dh_data <- readWorkbook(wb, sheet = "HMISFaacilities")

# Clean column names if needed
names(hc_data) <- trimws(names(hc_data))
names(dh_data) <- trimws(names(dh_data))

# Get the column name in Sheet 1 where the hospital should be filled
# (Assuming it's called "Correct DH Name", update if different)
target_col <- "Correct DH Name"  # <- UPDATE if needed

# Match logic (case-sensitive): Check if DH name is a prefix of HC name
for (i in seq_len(nrow(hc_data))) {
  hc_name <- hc_data[i, 1]

  # Find a matching DH name that is a prefix to the HC name
  match_idx <- which(sapply(dh_data[[1]], function(dh) {
    grepl(paste0("^", dh, " "), hc_name, fixed = FALSE)
  }))

  if (length(match_idx) > 0) {
    matched_dh <- dh_data[[1]][match_idx[1]]
    hc_data[i, target_col] <- matched_dh

    # Highlight that DH row in yellow
    addStyle(
      wb, sheet = "HMISFaacilities",
      style = createStyle(fgFill = "#FFFF00"),
      rows = match_idx[1] + 1, cols = 1,
      gridExpand = TRUE
    )
  }
}

# Overwrite Sheet 1 with updated data
writeData(wb, sheet = "TelMentorshipFacilities", x = hc_data, withFilter = TRUE)

# Save the updated file
saveWorkbook(wb, file = file_path, overwrite = TRUE)
